In [4]:
import numpy as np
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
from tslearn.metrics import dtw
from sklearn.preprocessing import StandardScaler

# ---------- Time Series & Subsequence Setup ----------
np.random.seed(42)
n = 300
t = np.linspace(0, 30, n)
ts = np.sin(t) + 0.3 * np.random.randn(n)
ts[120:130] += 5  # Spike anomaly
ts[200:205] -= 4  # Dip anomaly

window_size = 21
stride = 1

def extract_subsequences(ts, window_size=21, stride=1):
    subsequences = []
    indices = []
    for i in range(0, len(ts) - window_size + 1, stride):
        subsequences.append(ts[i:i + window_size])
        indices.append(i)
    return np.array(subsequences), np.array(indices)

subseqs, start_indices = extract_subsequences(ts, window_size, stride)
scaler = StandardScaler()
subseqs_scaled = scaler.fit_transform(subseqs)

# ---------- Precompute all pairwise DTW distances ----------
dtw_distances = []
pair_indices = []

for i in range(len(subseqs_scaled)):
    for j in range(i + 1, len(subseqs_scaled)):
        d = dtw(subseqs_scaled[i], subseqs_scaled[j])
        dtw_distances.append(d)
        pair_indices.append((i, j))

# ---------- Dash App Layout ----------
app = dash.Dash(__name__)
app.title = "DTW Graph-Based Anomaly Detection"

app.layout = html.Div([
    html.H2("Time Series Anomaly Detection Using DTW & Graph"),
    dcc.Graph(id='ts-plot'),
    html.Label("DTW Distance Threshold:"),
    dcc.Slider(
        id='dtw-threshold-slider',
        min=0.5, max=5.0, step=0.1, value=2.5,
        marks={i: str(i) for i in range(1, 6)}
    ),
    dcc.Graph(id='dtw-histogram'),
    html.Div(id='anomaly-stats', style={"marginTop": "20px", "fontFamily": "Courier New"})
])

# ---------- Callback to update plots ----------
@app.callback(
    [Output('ts-plot', 'figure'),
     Output('dtw-histogram', 'figure'),
     Output('anomaly-stats', 'children')],
    [Input('dtw-threshold-slider', 'value')]
)
def update_graphs(threshold):

    # Build Graph from DTW similarity
    import networkx as nx
    G = nx.Graph()
    num_nodes = len(subseqs)
    G.add_nodes_from(range(num_nodes))
    for (i, j), dist in zip(pair_indices, dtw_distances):
        if dist < threshold:
            G.add_edge(i, j, weight=dist)

    # Degree-based anomaly detection
    node_degrees = np.array([G.degree(n) for n in G.nodes()])
    deg_thresh = np.percentile(node_degrees, 10)
    anomaly_nodes = [n for n, d in enumerate(node_degrees) if d <= deg_thresh]
    anomaly_indices = start_indices[anomaly_nodes]

    # --- Time Series Plot ---
    ts_trace = go.Scatter(y=ts, mode='lines', name='Time Series')
    anomaly_spans = [
        go.Scatter(
            x=[idx, idx + window_size],
            y=[ts[idx], ts[idx + window_size - 1]],
            mode='lines',
            line=dict(width=0),
            fill='toself',
            fillcolor='rgba(255,0,0,0.3)',
            name='Anomaly'
        )
        for idx in anomaly_indices
    ]
    ts_fig = go.Figure([ts_trace] + anomaly_spans)
    ts_fig.update_layout(title="Detected Anomalies on Time Series")

    # --- Histogram of DTW Distances ---
    hist_fig = go.Figure()
    hist_fig.add_trace(go.Histogram(x=dtw_distances, nbinsx=50, name="DTW Distances"))
    hist_fig.add_vline(x=threshold, line_width=2, line_dash="dash", line_color="red")
    hist_fig.update_layout(title="Distribution of DTW Distances", xaxis_title="DTW Distance")

    # --- Stats Text ---
    anomaly_rate = 100 * len(anomaly_indices) / len(ts)
    stats_text = f"""
    📊 Stats:
    - Total subsequences: {num_nodes}
    - DTW threshold: {threshold}
    - Degree threshold (10th percentile): {deg_thresh:.2f}
    - Detected anomalies: {len(anomaly_indices)}
    - Anomaly coverage: {anomaly_rate:.2f}%
    """
    return ts_fig, hist_fig, stats_text

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


ModuleNotFoundError: No module named 'flask.typing'

In [3]:
# pip install dash